# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv


In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)



In [3]:
import os
from glob import glob

# Write your code below.
# load the price_data environment variable


price_data_dir = os.getenv("PRICE_DATA")



In [4]:
price_data_dir

'../../05_src/data/prices/'

In [5]:
parquet_files = glob(os.path.join(price_data_dir+"**/*.parquet/*.parquet"))
dd_px = dd.read_parquet(parquet_files).set_index("ticker")


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [6]:
# Add lags for Close and returns based on Close:
dd_px['Close_lag'] = dd_px['Close'].shift(1)
dd_px['returns'] = dd_px['Close'] / dd_px['Close_lag'] - 1 


In [7]:

# Add range hi_lo_range: 
dd_px['hi_lo_range'] = dd_px['High'] - dd_px['Low']


In [8]:

# Assign results to dd_feat
dd_feat = dd_px

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [9]:
# Write your code below.

import pandas as pd

#Convert dask dataframe into pandas dataframe
df_pd = dd_feat.compute()

# Add rolling average return calculation with a window of 10 days
df_pd['rolling_avg_return'] = df_pd.groupby('ticker')['returns'].rolling(10).mean().reset_index(level=0, drop=True)



In [10]:
dd_feat.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,returns,hi_lo_range
ticker,,,,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.613010,4674353,Health Care,Life Sciences Tools & Services,2000,NaN,NaN,8.270744
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.281460,4765083,Health Care,Life Sciences Tools & Services,2000,51.502148,-0.076389,2.950645
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.782818,5758642,Health Care,Life Sciences Tools & Services,2000,47.567955,-0.062030,4.425964
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.344162,2534434,Health Care,Life Sciences Tools & Services,2000,44.617310,-0.038076,2.771820
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.372856,2819626,Health Care,Life Sciences Tools & Services,2000,42.918453,0.083333,4.962444
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTS,2024-06-20,169.820007,171.100006,167.100006,168.179993,168.179993,3233700,Health Care,Pharmaceuticals,2024,169.660004,-0.008723,4.000000
ZTS,2024-06-21,169.380005,171.789993,169.350006,171.089996,171.089996,4899700,Health Care,Pharmaceuticals,2024,168.179993,0.017303,2.439987
ZTS,2024-06-24,171.710007,174.289993,170.300003,171.839996,171.839996,1885900,Health Care,Pharmaceuticals,2024,171.089996,0.004384,3.989990


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Dask provides its own implementation of rolling operations (rolling() method), which allows you to compute rolling statistics efficiently, especially when working with large datasets that don't fit into memory.
Advantages of Using Dask for Rolling Operations:

Lazy Evaluation: Dask operations are lazily evaluated, meaning computations are deferred until necessary (e.g., when results are needed for further processing or output). This allows Dask to handle datasets that are larger than memory by operating on chunks of data.

Parallelism: Dask can leverage parallelism across multiple cores or even distributed computing clusters, making it scalable for large datasets. This parallel execution is inherent in Dask's design, making it suitable for computationally intensive tasks like rolling calculations.

Integration with Dask DataFrame: Performing operations directly on a Dask DataFrame (dd_feat in this case) maintains the benefits of distributed computing and avoids the overhead of data transfer between Dask and pandas.

It would have been better to calculate the rolling average mean because of efficiency and scalability. Performing calculations in Dask avoids unnecessary data transfer and retains the advantages of distributed computing and lazy evaluation. Dask can handle larger datasets that exceed memory limits by operating on chunks of data and utilizing parallelism effectively.





## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.